In [1]:
import pandas as pd
from os import walk
from pathlib import Path

import pydicom
import nrrd

import numpy as np
from tqdm import tqdm

## Loading translation and encoding database.

The files of the experiment are anonymized, thus requiring a file to re-translate sed experiments.

In [5]:
translation = pd.read_excel("../databases/WI-EXP_new.xlsx")
translation.head()

,FolderName,SeriesDescription,Random,DataType,SequenceType,Randomisation_Number,Exp_match,Exp_kidney_number,Side,kidney_number,WI
0,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1447_T1_r_108min,Exvivo 7,ExVivo,T1,7,3,D830+D831,Right,D831,WI
1,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1448_T1_r_247min,Exvivo 7,ExVivo,T1,7,3,D830+D831,Right,D831,WI
2,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1446_T1_l_242min,Exvivo 7,ExVivo,T1,7,3,D830+D831,Left,D830,noWI
3,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1445_T1_l_103min,Exvivo 7,ExVivo,T1,7,3,D830+D831,Left,D830,noWI
4,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1419_T2_r_129min,Exvivo 7,ExVivo,T2,7,3,D830+D831,Right,D831,WI


In [4]:
translation[translation.SequenceType == "T2"]

,FolderName,SeriesDescription,Random,DataType,SequenceType,Randomisation_Number,Exp_match,Exp_kidney_number,Side,kidney_number,WI
4,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1419_T2_r_129min,Exvivo 7,ExVivo,T2,7,3,D830+D831,Right,D831,WI
5,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1416_T2_l_126min,Exvivo 7,ExVivo,T2,7,3,D830+D831,Left,D830,noWI
6,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1417_T2_l_190min,Exvivo 7,ExVivo,T2,7,3,D830+D831,Left,D830,noWI
7,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1420_T2_r_194min,Exvivo 7,ExVivo,T2,7,3,D830+D831,Right,D831,WI
8,Anonymized - 21.05.26-19-50-41-DST-1.3.12.2.11...,#1415_T2_l_10min,Exvivo 7,ExVivo,T2,7,3,D830+D831,Left,D830,noWI
...,...,...,...,...,...,...,...,...,...,...,...
823,Anonymized - 22.01.28-16-46-43-STD-1.3.12.2.11...,#2984_T2_r_142min,Exvivo 30,ExVivo,T2,30,33,D888+D889,Right,D889,WI
824,Anonymized - 22.01.28-16-46-43-STD-1.3.12.2.11...,#2979_T2_l_139min,Exvivo 30,ExVivo,T2,30,33,D888+D889,Left,D888,noWI
825,Anonymized - 22.01.28-16-46-43-STD-1.3.12.2.11...,#2977_T2_l_10min,Exvivo 30,ExVivo,T2,30,33,D888+D889,Left,D888,noWI
826,Anonymized - 22.01.27-07-53-50-STD-1.3.12.2.11...,#2963_T2_both,Invivo 30,InVivo,T2,30,33,D888+D889,Both,Invivo,Invivo


## encoding and formatting nrrds for T1, T2 and T2star

In [ ]:
experiments = ["T1/Exvivo", "T1/Invivo", "T2/Exvivo", "T2/Invivo", "T2star/Exvivo", "T2star/Invivo"]
out_dir = Path("../data/processed/nrrds/")

for experiment in experiments:
    for (root, directories, file) in tqdm(walk(f"../MRI_data/{experiment}")):
        if not root == "../MRI_data/" + experiment\
              and "Research Groningen" in root\
                and not root.endswith("Groningen"):
            sequence = Path(root)
            parts = sequence.parts
            exp = parts[4]
            time = parts[-1]
            folder_name = "_".join([exp, time])
            description = translation[translation["FolderName"] == folder_name].SeriesDescription.to_string().split("#")[-1]
            ischemia = translation[translation["FolderName"] == folder_name].WI.to_string().split()[-1]
            side = translation[translation["FolderName"] == folder_name].Side.to_string().split()[-1]
            kiney_number = translation[translation["FolderName"] == folder_name].kidney_number.to_string().split()[-1]
            experiment_n = translation[translation["FolderName"] == folder_name].Exp_match.to_string().split()[-1]
            name = '_'.join([description, ischemia, side, kiney_number, experiment_n,]) + ".nrrd"
            volume = []
            for image in Path(root).glob("*.dcm"):
                image = pydicom.dcmread(image).pixel_array
                volume.append(image)
            try:
                volume = np.array(volume).transpose((1, 2, 0))
                f_out = out_dir/Path(name)
                nrrd.write(str(f_out), volume)
            except ValueError:
                print("Empty volume encountered... Will be skipping fr fr")


## Formatting T1, T2, T2star

In [ ]:
out_dir = Path("../data/processed/nrrds/")
place = {"l": "left", "r": "right", "both": "both"}
for (root, dir, files) in walk(out_dir):
    for file in files:
        if not "DWI" in file and not "ASL" in file and not "Store" in file:
            file = Path(file)
            rename_file = out_dir/file
            stem = file.stem
            parts = stem.split("_")[1:]
            time = parts[2].strip("min")

            if not parts[3] == "Invivo":
                vivo = "exvivo"
            else:
                vivo = "invivo"
                time = "-10"
            placement = place[parts[1]]    
            experiment = parts[-1]
            kidney = parts[-2]
            ischemia = parts[3].lower()
            if ischemia == "Both":
                ischemia = "invivo"
            sequence = parts[0]
            file = '_'.join([sequence, experiment, time, vivo, placement, ischemia, kidney]) + ".nrrd"
            f_out = out_dir/Path(file)
            rename_file.rename(f_out)
            

## Adding encoding for DWI and ASL

In [ ]:
out_dir = Path("../data/processed/nrrds/")
for (root, dir, files) in walk(out_dir):
    for file in files:
        if file.startswith("DWI") or file.startswith("ASL"):
            file = Path(file)
            rename_file = out_dir/file
            stem = file.stem
            parts = stem.split("_")
            exp = int(parts[1])
            lr = parts[-1].capitalize()

            ischemia, kidney = translation[(translation.Exp_match == exp) & 
                                           (translation.Side == lr)]\
                                            [["WI", "kidney_number"]]\
                                            .drop_duplicates()\
                                                .values[0]
            f_out = out_dir/Path('_'.join(parts + [ischemia.lower(), kidney]) + ".nrrd")
            rename_file.rename(f_out)